# Aggregate flows into one record.  
(Aggregate on SrcAddr)

# 4colab

In [1]:
# # Load the Drive helper and mount
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

# %cd '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'
# !ls

# data_dir = '/content/drive/MyDrive/Colab Notebooks/C73_A2_1/data'

In [2]:
data_dir = 'data'

# Import

In [3]:
import pandas as pd
import numpy as np
# import datetime
import copy

import warnings
warnings.filterwarnings('ignore')


window_width = 7200 #10800 #120 #60 #600 # 120  # seconds
window_stride = 3600 #10800 #60 #60 #30 #600 # 60  # seconds

start_idx = 0
end_idx = 12000

# Read data

In [4]:
af = 'u'

dt = 'v'

path = f'{data_dir}/{dt}.{af}1.csv'
# path = f'{data_dir}/{dt}_sm.{af}1.csv'
df = pd.read_csv(path, index_col=[0])

In [5]:
df['Conversation'] = df['SrcAddr']+' -> '+df['DstAddr']

In [6]:
df.head()

,StreamID,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,...,Label,LabelStr,PktsPerSec,BytesPerSec,SrcBytesPerSec,BytesPerPkt,DstBytes,DstBytesPerSec,State_orig,Conversation
0,1,2022-07-25 23:29:01.134063,75.056648,tcp,77.128.111.246,80,<?>,150.35.89.190,55706,alltcp,...,0,flow=Background,107.798579,134163.411909,133766.764833,1244.574960,29771,396.647076,FPA_FA,77.128.111.246 -> 150.35.89.190
1,2,2022-07-25 23:29:01.134402,6.596284,tcp,150.35.87.167,22,<?>,86.211.196.126,15174,alltcp,...,3,flow=From-Normal-V44-Grill,4.851216,490.276057,231.493982,101.062500,1707,258.782075,FPA_FPA,150.35.87.167 -> 86.211.196.126
2,3,2022-07-25 23:29:01.135568,3574.257784,tcp,220.171.214.187,3953,<?>,150.35.87.17,22,alltcp,...,0,flow=Background,448.802828,444301.593533,10812.557554,989.970575,1549401561,433489.035979,PA_PA,220.171.214.187 -> 150.35.87.17
3,4,2022-07-25 23:29:01.142763,1341.171623,udp,176.12.135.158,54371,<->,150.35.87.62,15045,CON,...,0,flow=Background-Established-cmpgw-CVUT,211.289887,183526.597793,89585.377406,868.600958,125991299,93941.220387,CON,176.12.135.158 -> 150.35.87.62
4,5,2022-07-25 23:29:01.148062,3594.250928,tcp,91.106.43.120,37380,<?>,150.35.87.62,56306,alltcp,...,0,flow=Background-Established-cmpgw-CVUT,63.803559,59280.284614,57395.090415,929.106242,6775861,1885.194199,PA_PA,91.106.43.120 -> 150.35.87.62


# Divide Time Window

In [7]:
df['StartTimeInt'] = pd.to_datetime(df['StartTime']).astype(np.int64)*1e-9
datetime_start = df['StartTimeInt'].min()

df['Window_lower'] = (df['StartTimeInt'] - datetime_start - window_width) / window_stride + 1
df['Window_lower'].clip(lower=0, inplace=True)
df['Window_upper_excl'] = (df['StartTimeInt'] - datetime_start) / window_stride + 1
df = df.astype({'Window_lower': int, 'Window_upper_excl': int})

# df.sort_values(by=['Window_lower'])
df#.head()

,StreamID,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,...,BytesPerSec,SrcBytesPerSec,BytesPerPkt,DstBytes,DstBytesPerSec,State_orig,Conversation,StartTimeInt,Window_lower,Window_upper_excl
0,1,2022-07-25 23:29:01.134063,75.056648,tcp,77.128.111.246,80,<?>,150.35.89.190,55706,alltcp,...,134163.411909,133766.764833,1244.574960,29771,396.647076,FPA_FA,77.128.111.246 -> 150.35.89.190,1.658792e+09,0,1
1,2,2022-07-25 23:29:01.134402,6.596284,tcp,150.35.87.167,22,<?>,86.211.196.126,15174,alltcp,...,490.276057,231.493982,101.062500,1707,258.782075,FPA_FPA,150.35.87.167 -> 86.211.196.126,1.658792e+09,0,1
2,3,2022-07-25 23:29:01.135568,3574.257784,tcp,220.171.214.187,3953,<?>,150.35.87.17,22,alltcp,...,444301.593533,10812.557554,989.970575,1549401561,433489.035979,PA_PA,220.171.214.187 -> 150.35.87.17,1.658792e+09,0,1
3,4,2022-07-25 23:29:01.142763,1341.171623,udp,176.12.135.158,54371,<->,150.35.87.62,15045,CON,...,183526.597793,89585.377406,868.600958,125991299,93941.220387,CON,176.12.135.158 -> 150.35.87.62,1.658792e+09,0,1
4,5,2022-07-25 23:29:01.148062,3594.250928,tcp,91.106.43.120,37380,<?>,150.35.87.62,56306,alltcp,...,59280.284614,57395.090415,929.106242,6775861,1885.194199,PA_PA,91.106.43.120 -> 150.35.87.62,1.658792e+09,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255932,1255933,2022-07-27 02:24:40.820681,0.000143,tcp,68.55.208.148,53674,->,150.35.88.103,30966,alltcp,...,882901.819491,455465.224341,63.000000,61,427436.595150,S_RA,68.55.208.148 -> 150.35.88.103,1.658889e+09,25,27
1255933,1255934,2022-07-27 02:24:40.821362,0.000151,tcp,68.55.208.148,53679,->,150.35.88.103,30967,alltcp,...,833603.782767,430033.697459,126.000000,61,403570.085308,S_RA,68.55.208.148 -> 150.35.88.103,1.658889e+09,25,27
1255934,1255935,2022-07-27 02:24:40.821430,0.000366,tcp,68.55.208.148,53678,->,150.35.88.103,30964,alltcp,...,344492.437390,177714.352622,63.000000,61,166778.084768,S_RA,68.55.208.148 -> 150.35.88.103,1.658889e+09,25,27
1255935,1255936,2022-07-27 02:24:40.821849,0.000298,tcp,68.55.208.148,53681,->,150.35.88.103,30969,alltcp,...,423462.962961,218453.115813,126.000000,61,205009.847148,S_RA,68.55.208.148 -> 150.35.88.103,1.658889e+09,25,27


In [8]:
nb_windows = df['Window_upper_excl'].max()
# print(df['Label'].value_counts())
# print(df['Label'].unique())
# print()
print(df[df['Label'] == 5]['SrcAddr'].unique())
print(df[df['Label'] == 5]['Window_lower'].unique())

['150.35.87.168']
[ 0  4  9 18 22]


# Aggregate by `src_ip` in each time window and calculate features

In [9]:
df['Window_lower'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25])

In [10]:
def RU(df):
    if df.shape[0] == 1:
        return 1.0
    else:
        proba = df.value_counts()/df.shape[0]
        h = proba*np.log10(proba)
        return -h.sum()/np.log10(df.shape[0])

def top(df):
    return df.mode()[0]

In [11]:
def process_window(X, i):
    if len(df.loc[(df['Window_lower'] <= i) & (df['Window_upper_excl'] > i)]) == 0:
        return X

    # df.loc[(df['Window_lower'] <= i) & (df['Window_upper_excl'] > i)].to_csv(f'df/w/tr.{i}.__all_flows.csv')
    print('-----------------------\nProcess window: ', i)
    # print(df)

    """ All flows from this SrcAddr in this time window """
    gb_SrcAddr = df.loc[(df['Window_lower'] <= i) & (df['Window_upper_excl'] > i)].groupby(['Conversation','SrcAddr','DstAddr','State','Proto'])
    # gb_SrcAddr = df.loc[(df['Window_lower'] <= i) & (df['Window_upper_excl'] > i)].groupby('SrcAddr')

    r = {
        # 'StartTime': ['min', 'max'],
        'StreamID': ['unique'],

        # 'DstAddr': ['nunique', RU],
        'Sport': ['nunique', 'mean', 'std', 'max', 'median', RU],
        'Dport': ['nunique', 'mean', 'std', 'max', 'median', RU],
        
        # 'sTos': ['nunique', 'mean', 'std', 'max', 'median', RU],
        # 'dTos': ['nunique', 'mean', 'std', 'max', 'median', RU],
        
        # # 'pattern': ['unique', 'nunique', RU, top],
        # 'Dir': ['nunique', RU],
        # # 'State_orig': ['nunique', RU],
        # # 'State': ['nunique', RU],
        
        # # 'Service': ['unique', 'nunique', RU, top],
        # # 'Proto': ['unique', 'nunique', RU, top],
        # # 'sTosStr': ['unique', 'nunique', RU, top],
        # # 'dTosStr': ['unique', 'nunique', RU, top],

        # 'Dur': ['sum', 'mean', 'std', 'max', 'median', RU],
        
        'TotBytes': ['sum', 'mean', 'std', 'max', 'median', RU],
        'SrcBytes': ['sum', 'mean', 'std', 'max', 'median', RU],
        'DstBytes': ['sum', 'mean', 'std', 'max', 'median', RU],
        'TotPkts': ['sum', 'mean', 'std', 'max', 'median', RU],
        
        'PktsPerSec': ['mean', 'std', 'max', 'median', RU],
        'BytesPerSec': ['mean', 'std', 'max', 'median', RU],
        'SrcBytesPerSec': ['mean', 'std', 'max', 'median', RU],
        'DstBytesPerSec': ['mean', 'std', 'max', 'median', RU],
        'BytesPerPkt': ['mean', 'std', 'max', 'median', RU],
    }
    
    agg = gb_SrcAddr.agg(r)
    agg['Label'] = gb_SrcAddr['Label'].agg(lambda x: x.mode()[0])

    tmp = gb_SrcAddr.size().to_frame(name='n_flows').join(agg).reset_index().assign(window_id=i)
    X = X.append(tmp)
    del gb_SrcAddr

    return X

In [12]:
nb_windows = df['Window_upper_excl'].max()

print(nb_windows)
# end_idx = min(nb_windows, end_idx)
print(start_idx, end_idx)

27
0 12000


In [13]:
X = pd.DataFrame()
for i in range(start_idx, end_idx):
    X = process_window(X, i)

-----------------------
Process window:  0
-----------------------
Process window:  1
-----------------------
Process window:  2
-----------------------
Process window:  3
-----------------------
Process window:  4
-----------------------
Process window:  5
-----------------------
Process window:  6
-----------------------
Process window:  7
-----------------------
Process window:  8
-----------------------
Process window:  9
-----------------------
Process window:  10
-----------------------
Process window:  11
-----------------------
Process window:  12
-----------------------
Process window:  13
-----------------------
Process window:  14
-----------------------
Process window:  15
-----------------------
Process window:  16
-----------------------
Process window:  17
-----------------------
Process window:  18
-----------------------
Process window:  19
-----------------------
Process window:  20
-----------------------
Process window:  21
-----------------------
Process window:  2

In [14]:
#? prevent exceed memory
del df

In [15]:
X.columns = ['_'.join(x) if isinstance(x, tuple) else x for x in X.columns.ravel()]
X = X.rename(columns={'Label_': 'Label', 'SrcAddr_': 'SrcAddr', 'StartTime_min': 'StartTime_earliest', 'StartTime_max': 'StartTime_latest'})

In [16]:
X.columns

Index(['Conversation', 'SrcAddr', 'DstAddr', 'State', 'Proto', 'n_flows',
       'StreamID_unique', 'Sport_nunique', 'Sport_mean', 'Sport_std',
       'Sport_max', 'Sport_median', 'Sport_RU', 'Dport_nunique', 'Dport_mean',
       'Dport_std', 'Dport_max', 'Dport_median', 'Dport_RU', 'TotBytes_sum',
       'TotBytes_mean', 'TotBytes_std', 'TotBytes_max', 'TotBytes_median',
       'TotBytes_RU', 'SrcBytes_sum', 'SrcBytes_mean', 'SrcBytes_std',
       'SrcBytes_max', 'SrcBytes_median', 'SrcBytes_RU', 'DstBytes_sum',
       'DstBytes_mean', 'DstBytes_std', 'DstBytes_max', 'DstBytes_median',
       'DstBytes_RU', 'TotPkts_sum', 'TotPkts_mean', 'TotPkts_std',
       'TotPkts_max', 'TotPkts_median', 'TotPkts_RU', 'PktsPerSec_mean',
       'PktsPerSec_std', 'PktsPerSec_max', 'PktsPerSec_median',
       'PktsPerSec_RU', 'BytesPerSec_mean', 'BytesPerSec_std',
       'BytesPerSec_max', 'BytesPerSec_median', 'BytesPerSec_RU',
       'SrcBytesPerSec_mean', 'SrcBytesPerSec_std', 'SrcBytesPerSec_max'

In [17]:
""" std can be Nan if only one element """
X.fillna(-1, inplace=True)
X

,Conversation,SrcAddr,DstAddr,State,Proto,n_flows,StreamID_unique,Sport_nunique,Sport_mean,Sport_std,...,DstBytesPerSec_max,DstBytesPerSec_median,DstBytesPerSec_RU,BytesPerPkt_mean,BytesPerPkt_std,BytesPerPkt_max,BytesPerPkt_median,BytesPerPkt_RU,Label,window_id
0,100.103.150.82 -> 150.35.88.127,100.103.150.82,150.35.88.127,INT,udp,1,[51802],1,28706.0,-1.0,...,7748.074198,7748.074198,1.0,0.000000,-1.0,0.000000,0.000000,1.0,0,0
1,100.105.98.174 -> 150.35.89.186,100.105.98.174,150.35.89.186,CON,udp,1,[11517],1,13707.0,-1.0,...,147832.523675,147832.523675,1.0,128.000000,-1.0,128.000000,128.000000,1.0,0,0
2,100.110.177.53 -> 150.35.89.47,100.110.177.53,150.35.89.47,CON,udp,1,[7496],1,43468.0,-1.0,...,112250.218967,112250.218967,1.0,127.000000,-1.0,127.000000,127.000000,1.0,0,0
3,100.118.154.156 -> 150.35.89.102,100.118.154.156,150.35.89.102,CON,udp,1,[129864],1,14888.0,-1.0,...,96291.012055,96291.012055,1.0,128.000000,-1.0,128.000000,128.000000,1.0,0,0
4,100.120.103.93 -> 150.35.89.113,100.120.103.93,150.35.89.113,CON,udp,1,[60026],1,17995.0,-1.0,...,273.032536,273.032536,1.0,71.777778,-1.0,71.777778,71.777778,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,99.47.186.161 -> 150.35.88.59,99.47.186.161,150.35.88.59,alltcp,tcp,1,[1208451],1,80.0,-1.0,...,655460.141819,655460.141819,1.0,61.000000,-1.0,61.000000,61.000000,1.0,0,26
5461,99.47.186.161 -> 150.35.88.67,99.47.186.161,150.35.88.67,alltcp,tcp,1,[1198816],1,80.0,-1.0,...,0.000000,0.000000,1.0,0.000000,-1.0,0.000000,0.000000,1.0,0,26
5462,99.47.186.161 -> 150.35.89.22,99.47.186.161,150.35.89.22,alltcp,tcp,1,[1197498],1,80.0,-1.0,...,361079.327650,361079.327650,1.0,121.000000,-1.0,121.000000,121.000000,1.0,0,26
5463,99.47.186.161 -> 150.35.90.116,99.47.186.161,150.35.90.116,alltcp,tcp,1,[1222242],1,80.0,-1.0,...,0.000000,0.000000,1.0,0.000000,-1.0,0.000000,0.000000,1.0,0,26


# Save the processed data

In [18]:
# X.to_csv(f'{data_dir}/{dt}.{af}4.{window_width}-{window_stride}.w-{start_idx}-{end_idx}.__full__.csv')
X.to_csv(f'{data_dir}/{dt}.{af}4.{window_width}-{window_stride}.__full__.csv')

In [19]:
print(len(X))
X = X.loc[X['n_flows'] > 1]
print(len(X))
# X.to_csv(f'{data_dir}/{dt}.{af}4.{window_width}-{window_stride}.w-{start_idx}-{end_idx}.csv')
X.to_csv(f'{data_dir}/{dt}.{af}4.{window_width}-{window_stride}.csv')

295783
77907


In [20]:
#? prevent exceed memory
del X